#### Friday, November 1, 2024

[Quick and Dirty: Building a Private RAG Conversational Agent with LM Studio, Chroma DB, and LangChain](https://medium.com/@mr.ghulamrasool/quick-and-dirty-building-a-private-conversational-agent-for-healthcare-a-journey-with-lm-studio-f782a56987bd)

I will be implementing the code from [Local RAG Chatbot](https://github.com/grasool/Local-RAG-Chatbot) into this folder. 

mamba activate langchain-chroma


In [1]:
from openai import OpenAI
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

In [2]:
  # Point to the local server
# hermes-3-llama-3.1-8b
client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

In [3]:
embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


/tmp/ipykernel_57656/799427326.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/home/rob/miniforge3/envs/langchain-chroma/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
vector_db = Chroma(persist_directory="./chroma_db_nccn", embedding_function=embedding_function)

/tmp/ipykernel_57656/4087851065.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory="./chroma_db_nccn", embedding_function=embedding_function)


In [5]:
history = [
    {"role": "system", "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."},
    {"role": "user", "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."},
]

In [ ]:
while True:

    completion = client.chat.completions.create(
        model="local-model",
        messages=history,
        temperature=0.7,
        stream=True,
    )

    new_message = {"role": "assistant", "content": ""}
    
    for chunk in completion:
        if chunk.choices[0].delta.content:
            print(chunk.choices[0].delta.content, end="", flush=True)
            new_message["content"] += chunk.choices[0].delta.content

    history.append(new_message)
    
    #Uncomment to see chat history
    import json
    gray_color = "\033[90m"
    reset_color = "\033[0m"
    print(f"{gray_color}\n{'-'*20} History dump {'-'*20}\n")
    print(json.dumps(history, indent=2))
    print(f"\n{'-'*55}\n{reset_color}")

    print()
    next_input = input("> ")
    search_results = vector_db.similarity_search(next_input, k=2)
    some_context = ""
    for result in search_results:
        some_context += result.page_content + "\n\n"
    history.append({"role": "user", "content": some_context + next_input})

I am an intelligent assistant. I provide well-reasoned answers that are both correct and helpful.


-------------------- History dump --------------------

[
  {
    "role": "system",
    "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."
  },
  {
    "role": "user",
    "content": "Hello, introduce yourself to someone opening this program for the first time. Be concise."
  },
  {
    "role": "assistant",
    "content": "I am an intelligent assistant. I provide well-reasoned answers that are both correct and helpful.\n\n"
  }
]

-------------------------------------------------------



-------------------- History dump --------------------

[
  {
    "role": "system",
    "content": "You are an intelligent assistant. You always provide well-reasoned answers that are both correct and helpful."
  },
  {
    "role": "user",
    "content": "Hello, introduce yourself to someone opening this program for the first time. 